In [28]:
import pandas as pd
import os
import Levenshtein
import html
import PyPDF2
import datetime
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import sys
import fitz

In [2]:
input_file = open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/wegweiser_senioren.pdf','rb')

## PyPDF2 text extraction 

In [25]:
now = datetime.datetime.now()
#file01 = open(r'/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/output_file_pypdf1.txt','w')
with open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/wegweiser_senioren.pdf','rb') as pdf_file:
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    pypdf_test = ''
    for pg_no in range(number_of_pages): 
        page = read_pdf.getPage(pg_no)
        page_content = page.extractText()
        #file01.writelines(page_content)
        pypdf_test += page_content
        
        
#file01.close()        
stop = datetime.datetime.now()
pypdf_time = stop - now



## PyMuPDF text extraction

In [29]:
input_file1 = open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/wegweiser_senioren.pdf')
now = datetime.datetime.now()
#file02 = open(r'/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/output_file_pymupdf.txt','w')

doc = fitz.open(input_file1)  
text = ''
for page in doc:  
    text += page.get_text() 

pymupdf_test = text
#file02.writelines(pymupdf_test) 
#file02.close()

stop = datetime.datetime.now()
pymupdf_time = stop - now

## Using NLP method parser from Tika Module 

In [7]:
from tika import parser
now = datetime.datetime.now()
file_data = parser.from_file('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/wegweiser_senioren.pdf')
 
# file_data['content'] is used to get the content of the pdf file.

output = file_data['content']

# This output.encode encodes the text into utf-8 format.

output1 = output.encode('utf-8', errors='ignore')
 
# finally it saves the text file to a file called output.txt.

#with open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/output_file4546.txt', 'w') as the_file:
    #the_file.write(str(output1))
tika_time = stop - now

## Baseline text file (converted the given pdf to txt using online pdf converter)

In [9]:
now = datetime.datetime.now()

f = open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/wegweiser_senioren.txt', "r", encoding="utf8")
baseline = f.read()

stop = datetime.datetime.now()
base_time = stop - now

## Cosine Similarity 

In [10]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

def clean_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

corpi = [baseline,pypdf_test,pymupdf_test,output]

corpi = list(map(clean_string, corpi))

vectorizer = CountVectorizer().fit_transform(corpi)
vectors = vectorizer.toarray()

cosim = cosine_similarity(vectors)

def cosine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

print("Cosine similarity for PypDF2:")
print(cosine_sim_vectors(vectors[0], vectors[1]))
print("Cosine similarity for MuPdf:")
print(cosine_sim_vectors(vectors[0], vectors[2]))
print("Cosine similarity for tika:")
print(cosine_sim_vectors(vectors[0], vectors[3]))


Cosine similarity for PypDF2:
0.9999616482818724
Cosine similarity for MuPdf:
0.9999666704679816
Cosine similarity for tika:
0.9998491826986143


## TDF-idf Similarity 

In [11]:
def tfidf_string_similarity_test(correct_string,test_string):
    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    two_strings = [correct_string,test_string]
    vect = TfidfVectorizer(min_df=1, stop_words="english")
    tfidf = vect.fit_transform(two_strings)
    pairwise_similarity = tfidf * tfidf.T
    arr = pairwise_similarity.toarray()
    np.fill_diagonal(arr, -1)
    return arr[0][1]

In [12]:

print("Tfidf similarity for PyPDF2:")
print(tfidf_string_similarity_test(baseline, pypdf_test))
print("Tfidf similarity for PDFminer_six:")
print(tfidf_string_similarity_test(baseline, pymupdf_test))
print("Tfidf similarity for Tika:")
print(tfidf_string_similarity_test(baseline, output))

Tfidf similarity for PyPDF2:
0.9999711754667839
Tfidf similarity for PDFminer_six:
0.9999711754667839
Tfidf similarity for Tika:
0.9998676758240562


In [13]:
data = {'Time taken to extract the data ': [base_time, pypdf_time,pymupdf_time,tika_time],
        'Cosine similarity': [cosine_sim_vectors(vectors[0], vectors[0]), cosine_sim_vectors(vectors[0], vectors[1]), cosine_sim_vectors(vectors[0], vectors[2]), cosine_sim_vectors(vectors[0], vectors[3])],
        'Tf-idf similarity': [tfidf_string_similarity_test(baseline, baseline), tfidf_string_similarity_test(baseline, pypdf_test), tfidf_string_similarity_test(baseline, pymupdf_test),tfidf_string_similarity_test(baseline, output)]
        }
df = pd.DataFrame(data, index=['Baseline_string', 'PyPdf', 'PyMuPdf', 'Tika'])

In [14]:
df

,Time taken to extract the data,Cosine similarity,Tf-idf similarity
Baseline_string,0 days 00:00:00.001995,1.000000,1.000000
PyPdf,0 days 00:00:01.505429,0.999962,0.999971
PyMuPdf,0 days 00:00:00.109780,0.999967,0.999971
Tika,-1 days +23:59:35.247835,0.999849,0.999868


In [ ]:
### 1. The Tf-idf similarity is almost the same for PyPdf and PyMuPdf.
### 2. The Baseline_string is compared to itself hence the score is 1 (used for comparation with other methods)
### 3. The time take to convert the same file is faster in case PyMuPdf is the winner in this case.

### SequenceMatcher to compare the pdf

In [70]:
baseline_txt = open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/wegweiser_senioren.txt')
read_baseline = baseline_txt.readlines()
input_file_mupypdf = open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/output_file_pymupdf.txt')
read_mupypdf = input_file_mupypdf.readlines()
input_file_pypdf = open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Assignments/1-3-pdf-files/flyers/output_file_pypdf.txt')
read_pypdf = input_file_pypdf.readlines()

In [67]:
from difflib import SequenceMatcher
sequenceScore = SequenceMatcher(None, read_baseline, read_mupypdf)
check_list = sequenceScore.ratio()*100
check_list = round(check_list,1) 

In [69]:
print(f'The 2 textfiles are {check_list} similar')

The 2 textfiles are 74.1 similar


In [71]:
from difflib import SequenceMatcher
sequenceScore = SequenceMatcher(None, read_baseline, read_pypdf)
check_list = sequenceScore.ratio()*100
check_list = round(check_list,1) 

In [72]:
print(f'The 2 textfiles are {check_list} similar')

The 2 textfiles are 0.0 similar
